In [ ]:
import pandas as pd
empdata = pd.read_csv('chesterfield_25-08-2021_09-00-00 (3).csv', index_col=False, delimiter = ',')
empdata.head(200)

In [ ]:
import pandas as pd
import hashlib

def generate_pseudo_random_id(value):
    return hashlib.md5(str(value).encode()).hexdigest()

    #return str(int(time.time())) + str(random.randint(1, 1000))



file_path = 'chesterfield_25-08-2021_09-00-00 (3).csv'
df = pd.read_csv(file_path, header=None, names=['DateTime', 'Location', 'Name', 'Order', 'Order_Total_Amount', 'PaymentMethod', 'CardNumber'])


def order_price_cleaning(df):
    df['Order'] = df['Order'].str.split(",")
    df = df.explode('Order')
    df[['Product', 'Price']] = df['Order'].str.rsplit('-', n=1, expand=True)
    df[['Product_Name', 'Flavour']] = df['Product'].str.rsplit('-', n=1, expand=True)
    df[['Size','Product']] = df['Product'].str.split(n=1, expand=True)
    return df

df = order_price_cleaning(df)
df.drop(['Order'], axis=1, inplace=True)
df.drop(['CardNumber'], axis=1, inplace=True)


df['Product_id'] = df.apply(lambda row: generate_pseudo_random_id(str(row['Flavour']) + str(row['Size']) + str(row['Product'])+str(row['Price'])), axis=1)
df['Customer_id'] = df ['Name'].apply(lambda _: generate_pseudo_random_id(_))
df['Order_id'] = df.apply(lambda row: generate_pseudo_random_id(str(row['DateTime']) + str(row['Location']) + str(row['Name'])), axis=1)
df['Branch_id'] = df.apply(lambda row: generate_pseudo_random_id(str(row['Location']) + str(row['DateTime'])), axis=1)

df.drop(['Name'], axis=1, inplace=True)

df.to_csv('transformed_data_with_ids.csv', index=False)

empdata = pd.read_csv('transformed_data_with_ids.csv', index_col=False, delimiter=',')
empdata.head(300)


In [ ]:
import csv
from connection import open_database_connection, close_database_connection

try:
    connection, cursor = open_database_connection()

    if connection:
        print("Connected to the database!")

        with open('transformed_data_with_ids.csv', 'r') as csvfile:
            csvreader = csv.DictReader(csvfile)
            data = list(csvreader)

            for row in data:
                branch_id = row.get('Branch_id')
                branch_location = row.get('Location')
                order_id = row.get('Order_id')
                order_timestamp = row.get('DateTime')
                customer_id = row.get('Customer_id')
                total_spend = row.get ('Order_Total_Amount')
                payment_method = row.get ('PaymentMethod')
                product_id = row.get ('Product_id')
                product_name = row.get ('Product_Name')
                product_flavour = row.get ('Flavour')
                product_size = row.get('Size')
                price = row.get('Price')


                # Insert data into branches table
                if branch_id and branch_location:
                    try:
                        cursor.execute("INSERT INTO branches (branch_id, branch_location) VALUES (%s, %s) ON CONFLICT DO NOTHING",
                                       (branch_id, branch_location))
                        connection.commit()
                    except Exception as inner_ex:
                        connection.rollback()
                        print(f"Error inserting row into 'branches' table: {row}")
                        print(inner_ex)

                # Insert data into products table
                if product_id and product_name and product_flavour and product_size and price:
                    try:
                        cursor.execute("INSERT INTO products (product_id, product_name, product_flavour, product_size, price) VALUES (%s, %s, %s, %s, %s) ON CONFLICT DO NOTHING",
                                       (product_id, product_name, product_flavour, product_size, price))
                        connection.commit()
                    except Exception as inner_ex:
                        connection.rollback()
                        print(f"Error inserting row into 'products' table: {row}")
                        print(inner_ex)


                # Insert data into order details table
                if order_id and product_id:
                    try:
                        cursor.execute("INSERT INTO orders_details (order_id, product_id) VALUES (%s, %s)",
                                       (order_id, product_id))
                        connection.commit()
                    except Exception as inner_ex:
                        connection.rollback()
                        print(f"Error inserting row into 'orders' table: {row}")
                        print(inner_ex)

                #Insert data into orders table
                if order_id and order_timestamp and branch_id and customer_id and total_spend and payment_method:
                    try:
                        cursor.execute("INSERT INTO orders (order_id, order_timestamp, branch_id, customer_id, total_spend, payment_method) VALUES (%s, %s, %s, %s, %s, %s) ON CONFLICT DO NOTHING",
                                       (order_id, order_timestamp, branch_id, customer_id, total_spend, payment_method))
                        connection.commit()
                    except Exception as inner_ex:
                        connection.rollback()
                        print(f"Error inserting row into 'orders' table: {row}")
                        print(inner_ex)

            print("Data inserted successfully into 'branches', 'products', and 'orders' tables.")

except Exception as ex:
    print(f'Failed to insert data:', ex)


finally:
    close_database_connection(connection, cursor)


In [ ]:
import csv
from connection import open_database_connection, close_database_connection

try:
    connection, cursor = open_database_connection()

    if connection:
        print("Connected to the database!")

        with open('transformed_data_with_ids.csv', 'r') as csvfile:
            csvreader = csv.DictReader(csvfile)
            data = list(csvreader)

            for row in data:
                branch_id = row.get('Branch_id')
                order_id = row.get('Order_id')
                order_timestamp = row.get('DateTime')
                customer_id = row.get('Customer_id')
                total_spend = row.get ('Order_Total_Amount')
                payment_method = row.get ('PaymentMethod')

                if order_id and order_timestamp and branch_id and customer_id and total_spend and payment_method:
                    try:
                        cursor.execute("INSERT INTO orders (order_id, order_timestamp, branch_id, customer_id, total_spend, payment_method) VALUES (%s, %s, %s, %s, %s, %s) ON CONFLICT DO NOTHING",
                                       (order_id, order_timestamp, branch_id, customer_id, total_spend, payment_method))
                        connection.commit()
                    except Exception as inner_ex:
                        connection.rollback()
                        print(f"Error inserting row into 'orders' table: {row}")
                        print(inner_ex)

            print("Data inserted successfully into 'branches', 'products', and 'orders' tables.")

except Exception as ex:
    print(f'Failed to insert data:', ex)


finally:
    close_database_connection(connection, cursor)

In [ ]:
import csv
from connection import open_database_connection, close_database_connection

try:
    connection, cursor = open_database_connection()

    if connection:
        print("Connected to the database!")

        with open('transformed_data_with_ids.csv', 'r') as csvfile:
            csvreader = csv.DictReader(csvfile)
            data = list(csvreader)

            for row in data:
                product_id = row.get ('Product_id')
                order_id = row.get('Order_id')

                if order_id and product_id:
                    try:
                        cursor.execute("INSERT INTO orders_details (order_id, product_id) VALUES (%s, %s)",
                                       (order_id, product_id))
                        connection.commit()
                    except Exception as inner_ex:
                        connection.rollback()
                        print(f"Error inserting row into 'orders' table: {row}")
                        print(inner_ex)

            print("Data inserted successfully into 'branches', 'products', and 'orders' tables.")

except Exception as ex:
    print(f'Failed to insert data:', ex)

finally:
    close_database_connection(connection, cursor)